<a href="https://colab.research.google.com/github/Nitesh8998/ML_using_TensorFlow/blob/master/Assignment_4_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

Install and import all the necessary libraries for the assignment.

In [2]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd

tf.random.set_seed(1)

     |████████████████████████████████| 86.3MB 1.2MB/s 
     |████████████████████████████████| 4.3MB 25.6MB/s 
     |████████████████████████████████| 501kB 49.2MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


### **Importing the dataset**

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [7]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
train, val = train_test_split(train, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")
train.head()


323 train examples
81 validation examples
102 test examples


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
8,0.21124,12.5,7.87,0.0,0.5240,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
434,13.91340,0.0,18.10,0.0,0.7130,6.208,95.0,2.2222,24.0,666.0,20.2,100.63,15.17,11.7
176,0.07022,0.0,4.05,0.0,0.5100,6.020,47.2,3.5549,5.0,296.0,16.6,393.23,10.11,23.2
498,0.23912,0.0,9.69,0.0,0.5850,6.019,65.3,2.4091,6.0,391.0,19.2,396.90,12.92,21.2
279,0.21038,20.0,3.33,0.0,0.4429,6.812,32.2,4.1007,5.0,216.0,14.9,396.90,4.85,35.1


Converting the Pandas DataFrames into Tensorflow Datasets

In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [9]:
batch_size = 32
train_ds = df_to_dataset(train, True, batch_size)
val_ds = df_to_dataset(val, False, batch_size)
test_ds = df_to_dataset(test, False, batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Defining Feature Columns

In [45]:
# define feature_columns as a list of features using functions from tf.feature_column
feature_columns = []
for feature_batch, label_batch in train_ds.take(1):
  A = list(feature_batch.keys())
print(A)
RAD = tf.feature_column.numeric_column("RAD")
rad_buckets = tf.feature_column.bucketized_column(RAD, boundaries=[2,5])
for i in A:
    feature_columns.append(tf.feature_column.numeric_column(i))

feature_columns.append(rad_buckets)
print(feature_columns)


['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TAX', shape=(1,)

### Building the model

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Model should contain following layers:

```
feature_layer

Dense(1, activation=None)
```

Use 'Adam' optimizer

Use 'mse' as your loss and metric

In [0]:
# Build and compile your model in this cell.
model = tf.keras.Sequential([
    feature_layer,
    #tf.keras.layers.Dense(4, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(1, activation=None)
])# Build and compile your model in this cell. Make sure to first answer Q2.

ptimizer = tf.keras.optimizers.Adam(0.001)

model.compile(loss='mse',
            optimizer='adam',
            metrics=['mse'])



In [48]:
model.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 64ms/step - loss: 62592.8089 - mse: 64102.8164 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 59042.4648 - mse: 59139.6328 - val_loss: 53076.7161 - val_mse: 52354.6094
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 54393.3409 - mse: 54451.6602 - val_loss: 48822.6797 - val_mse: 48126.3906
Epoch 4/200
11/11 [==============================] - 0s 4ms/step -

In [0]:
loss, mse = model.evaluate(test_ds)
print("Mean Squared Error - Test Data", mse)